# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [218]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /home/brunom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/brunom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/brunom/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [219]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [156]:
# load data from database
engine = create_engine('sqlite:///data/disasterResponse.db')
df = pd.read_sql('SELECT * FROM messages_categories', engine)

In [157]:
df.head()

,id,message,original,genre,aid_centers,aid_related,buildings,child_alone,clothing,cold,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [178]:
# preliminary cleaning
df.iloc[:,4:] = df.iloc[:,4:].astype('int64')
df.drop('related', axis=1, inplace=True)

In [252]:
#def pipeline_data(message):
#    df = pd.DataFrame(data=message)
#    df.iloc[:,4:] = df.iloc[:,4:].astype('int64')

In [ ]:
#df = pd.read_sql('SELECT * FROM messages_categories', engine, chunksize=1)
#    c.execute('SELECT * FROM big_table') 
#for row in c:
    # do_stuff_with_row

### 2. Write a tokenization function to process your text data

In [187]:
stop_words = set(stopwords.words('english'))
def tokenize(sentence):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    return [lemmatizer.lemmatize(word).lower().strip() for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

message = df.message.apply(lambda x: " ".join(tokenize(x)))

In [214]:
message[:10]

0       weather update cold front cuba could pas haiti
1                                         is hurricane
2                                 looking someone name
3    un report leogane destroyed only hospital st c...
4       say west side haiti rest country today tonight
5                          information national palace
6                             storm sacred heart jesus
7                 please need tent water we silo thank
8                   i would like receive message thank
9    i croix de bouquets we health issue they worke...
Name: message, dtype: object

#### Classes are very imbalanced

In [217]:
print(df.shape)
df.iloc[:,4:].apply(lambda x: x.value_counts())

(26216, 39)


,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,25907,15356,24883,26216.0,25811,25686,25022,21141,23761,25684,...,21742,25492,25745,23902,26096,23773,26057,25015,24544,18919
1,309,10860,1333,NaN,405,530,1194,5075,2455,532,...,4474,724,471,2314,120,2443,159,1201,1672,7297


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [182]:
# test method
message = df.message.apply(lambda x: " ".join(tokenize(x)))
vectorizer = CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(message).toarray()

X = vector_features 
y = df.iloc[:,4:].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32) # training

clf = MultiOutputClassifier(AdaBoostClassifier()) # training
clf.fit(X_train, y_train) # training
predictions = clf.predict(X_test)
[f1_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

KeyboardInterrupt: 

In [194]:
class Clean_text(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X=None, y=None):
        return self
    
    def transform(self, X, y=None):
        
        return pd.Series(X).apply(lambda x: " ".join(self.tokenize(x))).values
    
    def tokenize(self, sentence):
        #stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        return [lemmatizer.lemmatize(word).lower().strip() for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [195]:
clean_text = Clean_text()
clean_text.transform(df.message)[:5]

array(['weather update cold front cuba could pas haiti', 'is hurricane',
       'looking someone name',
       'un report leogane destroyed only hospital st croix functioning needs supply desperately',
       'say west side haiti rest country today tonight'], dtype=object)

In [204]:
ml_pipeline = Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [205]:
X = df.message
y = df.iloc[:,4:].values 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32) # training
X_train.shape, y_train.shape
ml_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('clean_text', Clean_text()), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

#### Because classes are imbalanced, accuracy is not a good metric for this problem.

In [220]:
preds = ml_pipeline.predict(X_test)
print([f1_score(y_test[:,column],predictions[:,column]) for column in range(0,len(df.iloc[:,4:].columns)-1)])

[0.06956521739130435, 0.725133214920071, 0.38167938931297707, 0.0, 0.3619047619047619, 0.308641975308642, 0.3683492496589359, 0.5778321108394459, 0.6836158192090396, 0.2965779467680608, 0.11023622047244094, 0.5123966942148761, 0.6385714285714286, 0.11214953271028037, 0.2916235780765254, 0.4270531400966184, 0.4261437908496732, 0.46435100548446073, 0.16666666666666666, 0.3142857142857143, 0.0, 0.40441683919944793, 0.2693997071742314, 0.28903225806451616, 0.27756653992395436, 0.6450676982591877, 0.26506024096385544, 0.05555555555555556, 0.5534382767191384, 0.0, 0.5707509881422924, 0.0, 0.314540059347181, 0.567391304347826]


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'scaler__with_mean': [True, False]
    'clf__kernel': ['linear', 'rbf'],
    'clf__C':[1, 10]
}
parameters = {'learning_rate': np.arange(0.1,0.5,0.1),
                  'n_estimators':[50,100]}

grid_clf = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

## Scrap Book

In [250]:
# test y straight from df
vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
final_features = pd.DataFrame(vectorizer.fit_transform(df_clean_message).toarray(), 
                              columns=vectorizer.get_feature_names())

X = vector_features
y2 = MultiLabelBinarizer().fit_transform(labels)
y = df.iloc[:,4:].values
print(type(y2[0][0]))
print(type(y[0][0]))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)
print(y_train.shape)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size=0.2, random_state=32)
print(y_train2.shape)
#clf = OneVsRestClassifier(SVC(probability=True))
#clf = OneVsRestClassifier(BernoulliNB())
clf = MultiOutputClassifier(BernoulliNB())
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

<class 'numpy.int64'>
<class 'numpy.int64'>
(20972, 36)
(20972, 35)


/home/brunom/.miniconda2/lib/python3.6/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


[0.9837909992372235,
 0.7688787185354691,
 0.918001525553013,
 1.0,
 0.9780701754385965,
 0.9702517162471396,
 0.9328756674294432,
 0.814836003051106,
 0.933257055682685,
 0.9687261632341724,
 0.9858886346300534,
 0.8983600305110603,
 0.9109458428680397,
 0.9832189168573608,
 0.8785278413424866,
 0.881578947368421,
 0.9248665141113653,
 0.9473684210526315,
 0.9837909992372235,
 0.9586193745232647,
 0.9921815408085431,
 0.8236079328756675,
 0.914187643020595,
 0.90255530129672,
 0.9363081617086194,
 0.7858504958047292,
 0.8773836765827613,
 0.9496567505720824,
 0.9725400457665904,
 0.906559877955759,
 0.9937070938215103,
 0.8991228070175439,
 0.9916094584286804,
 0.9204805491990846,
 0.9294431731502669]

#### Option 1 - use Tfidf

In [208]:
vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
final_features = pd.DataFrame(vectorizer.fit_transform(df_clean_message).toarray(), 
                              columns=vectorizer.get_feature_names())

display(final_features.head())

X = vector_features
y = MultiLabelBinarizer().fit_transform(labels)
print(len(labels))
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

#clf = OneVsRestClassifier(SVC(probability=True))
clf = OneVsRestClassifier(BernoulliNB())
#clf = OneVsRestClassifier(AdaBoostClassifier())
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

,aa,aai,ab,ababa,abad,abandon,abba,abdomin,abdomin pain,abduct,...,zhejiang provinc,zhouqu,zimbabw,zimbabwean,zinc,zinc roof,zone,zoonot,zoonot diseas,zy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


26216
(26216, 35)


[0.9855524734165511,
 0.7610957004160888,
 0.926490984743412,
 0.9808136846971799,
 0.9737632917244567,
 0.9379334257975035,
 0.8206195099398983,
 0.9248728617660656,
 0.9730698104484512,
 0.9880952380952381,
 0.9010633379565418,
 0.9054553860379103,
 0.9849745723532132,
 0.8839574664817383,
 0.8861534905224225,
 0.9290337494220989,
 0.9545769764216366,
 0.9845122515025427,
 0.9637078132223763,
 0.9931807674526121,
 0.8305594082293112,
 0.9218677762367082,
 0.9082293111419325,
 0.9467175219602404,
 0.7894128525196487,
 0.8769070735090153,
 0.9549237170596394,
 0.973994452149792,
 0.9060332871012483,
 0.9937586685159501,
 0.9008321775312067,
 0.9921405455386038,
 0.9239482200647249,
 0.929842810910772,
 0.8413083680073972]

#### Option 2: use CountVectorizer

In [203]:
vectorizer = CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
final_features = pd.DataFrame(vectorizer.fit_transform(df_clean_message).toarray(), 
                              columns=vectorizer.get_feature_names())

display(final_features.head())

X = vector_features
y = MultiLabelBinarizer().fit_transform(labels)
print(len(labels))
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

#clf = OneVsRestClassifier(SVC(probability=True))
clf = OneVsRestClassifier(BernoulliNB())
#clf = OneVsRestClassifier(AdaBoostClassifier())
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

,aa,aai,ab,ababa,abad,abandon,abba,abdomin,abdomin pain,abduct,...,zhejiang provinc,zhouqu,zimbabw,zimbabwean,zinc,zinc roof,zone,zoonot,zoonot diseas,zy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


26216
(26216, 35)


[0.9855524734165511,
 0.7610957004160888,
 0.926490984743412,
 0.9808136846971799,
 0.9737632917244567,
 0.9379334257975035,
 0.8206195099398983,
 0.9248728617660656,
 0.9730698104484512,
 0.9880952380952381,
 0.9010633379565418,
 0.9054553860379103,
 0.9849745723532132,
 0.8839574664817383,
 0.8861534905224225,
 0.9290337494220989,
 0.9545769764216366,
 0.9845122515025427,
 0.9637078132223763,
 0.9931807674526121,
 0.8305594082293112,
 0.9218677762367082,
 0.9082293111419325,
 0.9467175219602404,
 0.7894128525196487,
 0.8769070735090153,
 0.9549237170596394,
 0.973994452149792,
 0.9060332871012483,
 0.9937586685159501,
 0.9008321775312067,
 0.9921405455386038,
 0.9239482200647249,
 0.929842810910772,
 0.8413083680073972]

In [189]:
y.shape

(26216, 35)

In [128]:
#X = vector_features
y2 = df.iloc[:,4:-1].values
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.33, random_state=32)

In [163]:
def tokenize(sentence):
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    return [word for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 
df_clean_message = df.message.apply(lambda x: " ".join(tokenize(x)))
#vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l1', ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
X = vector_features
y = MultiLabelBinarizer().fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

#clf = OneVsRestClassifier(SVC(probability=True))
#clf = OneVsRestClassifier(BernoulliNB())
clf = OneVsRestClassifier(AdaBoostClassifier())
clf.fit(X_train, y_train)
clf.fit(X_train, y_trainb

accuracy_score(y_test, predictions)

0.2311604253351826

In [173]:
df.iloc[:,4:].sum(axis=0)

aid_centers                 309
aid_related               10860
buildings                  1333
child_alone                   0
clothing                    405
cold                        530
death                      1194
direct_report              5075
earthquake                 2455
electricity                 532
fire                        282
floods                     2155
food                       2923
hospitals                   283
infrastructure_related     1705
medical_help               2084
medical_products           1313
military                    860
missing_people              298
money                       604
offer                       118
other_aid                  3446
other_infrastructure       1151
other_weather              1376
refugees                    875
related                   20282
request                    4474
search_and_rescue           724
security                    471
shelter                    2314
shops                       120
storm   

In [175]:
len(df.iloc[:,4:].columns)

36

In [182]:
accuracy_score(y_test[:,0], predictions[:,0])

0.9878640776699029

In [184]:
y_test.shape

(8652, 35)

In [190]:
len(df.iloc[:,4:].columns)
[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

[0.9878640776699029,
 0.7457235321312992,
 0.9617429496070273,
 0.9872861766065649,
 0.9843966712898752,
 0.9664817383263985,
 0.8542533518261674,
 0.9696024040684235,
 0.9812760055478502,
 0.9912159038372631,
 0.9555016181229773,
 0.9435968562182154,
 0.9864771151178918,
 0.9294960702727693,
 0.9254507628294036,
 0.9526121128062875,
 0.9699491447064262,
 0.9878640776699029,
 0.9769995376791494,
 0.9939898289412853,
 0.8646555709662506,
 0.9520342117429496,
 0.9484512251502543,
 0.9684466019417476,
 0.7786638927415627,
 0.8919325011558021,
 0.970873786407767,
 0.9823162274618585,
 0.9469486823855756,
 0.994914470642626,
 0.9348127600554785,
 0.9942209893666204,
 0.9586222838650024,
 0.9605871474803513,
 0.8788719371243643]

In [175]:
clf = OneVsRestClassifier(MultinomialNB())
clf.fit(X_train, y_train)
preds = clf.predict(X_train) 

ValueError: Multioutput target data is not supported with label binarization

In [ ]:
OneVsRestClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X) 
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = BernoulliNB(alpha=1.0, class_prior=None, fit_prior=True)
clf.fit(X_train, y_train)


ValueError: bad input shape (17564, 37)

## Code Bank

In [32]:
# beautiful solution
#df.message.apply(lambda x: " ".join([stemmer.stem(word) for word in re.sub("[^a-zA-Z]", " ", x).split() 
#          if word not in stop_words]) )

In [42]:
class Select_features(BaseEstimator, TransformerMixin):
    # constructor
    def __init__(self, column_names):
        self.column_names = column_names 
    
    # pipeline must implement fit
    def fit(self, X=None, y=None):
        return self
    
    # return new dataframe with selected features
    def transform(self, X=None, y=None):
        return X[self.column_names] 

In [ ]:
X = vector_features
#y = MultiLabelBinarizer().fit_transform(labels)
#print(len(labels))
#print(y.shape)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)
#
#vectorizer = CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))
#clf = OneVsRestClassifier(BernoulliNB())
#message_pipeline = Pipeline([('vectorizer', vectorizer),('clf',clf)])
#message_pipeline.fit()
#message_pipeline.named_steps['vectorizer'].transform([df_clean_message])